In [ ]:
!pip install xml_to_dict #https://github.com/xthehatterx/xml_to_dict
!pip install sentencepiece
!pip install konlpy
!pip install transformers

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

import random
from dataclasses import dataclass
from typing import Dict, List, Optional, Union

In [ ]:
xml_sample_loc = 'xml_sample_20230519.csv'

In [ ]:
xml_sample = pd.read_csv(xml_sample_loc)
keyword = xml_sample['keyword']

In [ ]:
#get 중복 없는 keyword
keyword_unique = []

for i in range(len(keyword)):
  keyword_unique+=keyword[i].split('|')

keyword_unique = list(set(keyword_unique))

print(len(keyword_unique) )
print(keyword_unique[:20])

In [ ]:
#json파일에서 text들을 가져와 text_sum에 저장
import json
import os

text_sum = []

folder_path = "data/"  # 데이터 폴더 경로

# 폴더 내의 파일 목록 가져오기
file_list = os.listdir(folder_path)

for file_name in file_list:
    if file_name.endswith(".json"):  # .json 파일인 경우에만 처리
        file_path = os.path.join(folder_path, file_name)  # 파일 경로 생성

    try:
      with open(file_path, "r") as file_:
          data = json.load(file_)

      for i in range(len(data['form'])):
          if(data['form'][i]['text']!=None):
            text_sum.append(data['form'][i]['text'])  # 요소 추가

    except:
      print(f"error in {file_path}")


print(len(text_sum))
print(text_sum[0])

In [ ]:
train_tokenization_data = np.concatenate((text_sum,keyword_unique)) #text와 keyword 합쳐서 한번에 train

print(len(train_tokenization_data))

def get_training_corpus(): #개수가 많으므로 1000개씩 끊어서 return
    return (
        train_tokenization_data[i : i + 1000]
        for i in range(0, len(train_tokenization_data), 1000)
    )

training_corpus = get_training_corpus()

In [ ]:
old_tokenizer = AutoTokenizer.from_pretrained("KETI-AIR/ke-t5-base-ko") #pre-trained된거 불러오기
old_tokenizer.vocab_size #64100

In [ ]:
tokenizer = old_tokenizer.train_new_from_iterator(training_corpus, 15000) #뒤 숫자는 우리 데이터에서 단어 개수
tokenizer.vocab_size #14602
tokenizer.save_pretrained("tokenizer_finetuned_ket5_by_xml_data") #저장